1. Read PDF

In [66]:
from pypdf import PdfReader
import os

FILE_PATH = os.path.join("data","Lecture1-a.pdf")
reader = PdfReader(FILE_PATH)
number_of_pages = len(reader.pages)

entire_text = ""
for page_num in range(number_of_pages):
    page = reader.pages[page_num]
    entire_text += page.extract_text()

#entire_text[:200]

2. Split text into chunks 

In [3]:
# split text based on number of characters in the text
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [4]:
text_chunks = text_splitter.split_text(entire_text)
print(f"Total chunks: {len(text_chunks)}")

Total chunks: 15


In [5]:
# print the first two chunks
text_chunks[:2]

['Intro to DevOps and Beyond\nRavindu Nirmal FernandoAbout Me\n• STL - DevOps @ Sysco LABS - Sri Lanka\n• MSc in Computer Science specialized in \nCloud Computing (UOM)\n• AWS Certified Solutions Architect - \nProfessional \n• Certified Kubernetes Administrator \n(CKA)\n• AWS Community Builder\nRavindu Nirmal Fernando\nhttps://ravindunfernando.com\nThe Era before \nDevOpsDevelopers\nFocused on Agility\nOperators\nFocused on StabilityAct 01 - Operations teams \nmaintaining large fragile \napplications',
 "Operators\nFocused on StabilityAct 01 - Operations teams \nmaintaining large fragile \napplications\nDoesn't have any visibility on the \napplication, whether or not its \nworking as expected\nAct 03 - The Developers\nDevelopers taking shortcuts and \nputting more and more fragile \ncode on top of existing ones \nAct 02 - The product \nmanagers\nLarger, unrealistic commitments \nmade to the outside world (client/ \ninvestors) without understanding \nthe complexities behind \ndevelopmen

3. Embedding Chunks

In [6]:
# huggingface embeddings models lot of them available there
import torch
from sentence_transformers import SentenceTransformer

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "BAAI/bge-small-en-v1.5"
# model_name = "all-MiniLM-L6-v2"

embedding_model = SentenceTransformer(model_name, device=device)

d:\Education\LearningAssistant-ChatBot\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
embeddings = embedding_model.encode(text_chunks, show_progress_bar=True)

Batches: 100%|██████████| 1/1 [00:09<00:00,  9.77s/it]


In [ ]:
embeddings[0].shape # store the embeddings in a list dimension

(384,)

4. Store in the Vector Database

In [9]:
# Import client library
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient("http://localhost:6333")

In [10]:
from qdrant_client import QdrantClient

# Connect to Qdrant inside Docker (default port 6333)
client = QdrantClient(host="localhost", port=6333)

# Delete the collection
client.delete_collection(collection_name="qa_index")

True

In [11]:
# embedding_model.get_sentence_embedding_dimension()
collection_name = "qa_index"
client.delete_collection(collection_name)

client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
    
)
# demention of vector is 384
# create a collection in qdrant
# metrix is cosine for semantic similarity
# if return true collection is created we can stroe vectors in it


True

5. Create payloads and ids

In [12]:
ids = [] # list of ids for each vectors we can use to retrieve the vectors text chunks
payload = [] # metadata for each vector we can use to retrieve the text chunks

for id, text in enumerate(text_chunks):
    ids.append(id)
    payload.append({"source": FILE_PATH, "content": text})

payload[0]

{'source': 'data\\Lecture1-a.pdf',
 'content': 'Intro to DevOps and Beyond\nRavindu Nirmal FernandoAbout Me\n• STL - DevOps @ Sysco LABS - Sri Lanka\n• MSc in Computer Science specialized in \nCloud Computing (UOM)\n• AWS Certified Solutions Architect - \nProfessional \n• Certified Kubernetes Administrator \n(CKA)\n• AWS Community Builder\nRavindu Nirmal Fernando\nhttps://ravindunfernando.com\nThe Era before \nDevOpsDevelopers\nFocused on Agility\nOperators\nFocused on StabilityAct 01 - Operations teams \nmaintaining large fragile \napplications'}

In [13]:
# store the vectors in qdrant 
client.upload_collection(
    collection_name=collection_name,
    vectors=embeddings,
    payload=payload,
    ids=ids,
    batch_size=256,  # How many vectors will be uploaded in a single request?
)

In [14]:
# count the number of vectors in the collection
client.count(collection_name)

CountResult(count=15)

6. Retrieval Component

In [15]:
def search(text: str, top_k: int):# search for the text in the collection
    query_embedding = embedding_model.encode(text).tolist()
    
    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        query_filter=None,  
        limit=top_k
    )
    return search_result

In [60]:
# Retrieve the top 5 most similar vectors to the query
question = "what is object oriented programming?"
results = search(question, top_k=5) # retrieve the top 5 most similar vectors to the query
results

C:\Users\Vishwa\AppData\Local\Temp\ipykernel_8876\853163018.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(


[ScoredPoint(id=3, version=0, score=0.6344266, payload={'source': 'data\\Lecture1-a.pdf', 'content': 'at high velocity”\n- What is DevOps? [AWS] -\n“A compound of development (Dev) and operations (Ops), DevOps is the \nunion of people, process, and technology to continually provide value to \ncustomers.” \n- What is DevOps? [Azure] -\nDevOps allows evolving and improving products at a faster pace than businesses \nusing traditional software development and infrastructure management processes. \nThis speed allows businesses to serve their customers better and compete effectively.Reduce Organizational'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=2, version=0, score=0.63112485, payload={'source': 'data\\Lecture1-a.pdf', 'content': 'investors) without understanding \nthe complexities behind \ndevelopment and operations\nAct 04 - Dev and Ops at war\n"It worked on my machine" \nphenomenon \n"Destructive downward spiral in IT" - Gene Kim\nHow can we \novercome \nthese is

In [53]:
text_chunks[2]

'investors) without understanding \nthe complexities behind \ndevelopment and operations\nAct 04 - Dev and Ops at war\n"It worked on my machine" \nphenomenon \n"Destructive downward spiral in IT" - Gene Kim\nHow can we \novercome \nthese issues?“DevOps is the combination of cultural philosophies, practices, and tools \nthat increases an organization’s ability to deliver applications and services \nat high velocity”\n- What is DevOps? [AWS] -'

7. Response Generation

In [61]:
system_prompt = """You are an ai assistant for question-answering tasks. Answer the question according only to the given context.
If question cannot be answered using the context, simply say I don't know. Do not make stuff up.

Context: {context}
"""

user_prompt = """
Question: {question}

Answer:"""

references = [obj.payload["content"] for obj in results]


context = "\n\n".join(references)

Method 1: Using llama_index library (llama_index.llms.ollama) Version

In [65]:
from llama_index.llms.ollama import Ollama

final_prompt = system_prompt.format(context=context) + user_prompt.format(question=question)

# Step 4: Generate response using Ollama
llm = Ollama(model="gemma3:1b", temperature=0.1)
response = llm.complete(final_prompt)

# Step 5: Print answer and references
print(f"ANSWER:\n{response.text}\n")
print(f"REFERENCES:\n")
for index, ref in enumerate(references):
    print(f"Reference [{index + 1}]: {ref}\n")

ANSWER:
I don't know.

REFERENCES:

Reference [1]: at high velocity”
- What is DevOps? [AWS] -
“A compound of development (Dev) and operations (Ops), DevOps is the 
union of people, process, and technology to continually provide value to 
customers.” 
- What is DevOps? [Azure] -
DevOps allows evolving and improving products at a faster pace than businesses 
using traditional software development and infrastructure management processes. 
This speed allows businesses to serve their customers better and compete effectively.Reduce Organizational

Reference [2]: investors) without understanding 
the complexities behind 
development and operations
Act 04 - Dev and Ops at war
"It worked on my machine" 
phenomenon 
"Destructive downward spiral in IT" - Gene Kim
How can we 
overcome 
these issues?“DevOps is the combination of cultural philosophies, practices, and tools 
that increases an organization’s ability to deliver applications and services 
at high velocity”
- What is DevOps? [AWS] -

Re

Answer general LLM knowledge responses

In [48]:
import requests
import json

# 1. Format system and user messages
system_message = system_prompt.format(context=context)
user_message = user_prompt.format(question=question)

# 2. API URL
api_url = "http://localhost:11434/api/chat"

# 3. Payload
payload = {
    "model": "gemma3:1b",
    "messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ],
    "stream": True,   # <-- Tell Ollama to stream
    #"temperature": 0.1  # Lower temperature for more factual responses
}

# 4. Send request with streaming
response = requests.post(api_url, json=payload, stream=True)

# 5. Read the response stream
if response.status_code == 200:
    for line in response.iter_lines():
        if line:
            data = line.decode('utf-8')
            chunk = json.loads(data)
            if 'message' in chunk and 'content' in chunk['message']:
                print(chunk['message']['content'], end='', flush=True)
else:
    print(f"Error: {response.status_code} - {response.text}")


Object-oriented programming (OOP) is a programming paradigm that structures software design around data – called objects – and the concepts that relate to those objects. It’s based on the idea that real-world objects have properties (data) and behaviors (actions) that are encapsulated within them.  Key concepts include:

*   **Encapsulation:** Bundling data and methods that operate on that data within a single unit (an object).
*   **Inheritance:** Creating new objects based on existing objects, inheriting their properties and methods.
*   **Polymorphism:** The ability of objects to take on different forms.

Essentially, OOP is about modeling real-world entities as code, making it easier to manage and reuse code.

8. Response with References

RAG (Retrieval-Augmented Generation) principles: discourage hallucinations, only answer from the given documents

Method 2: Using direct API requests

In [ ]:
import requests
import json

# 1. Format prompt
final_prompt = system_prompt.format(context=context) + user_prompt.format(question=question)

# 2. API URL for completion mode
api_url = "http://localhost:11434/api/generate"

# 3. Payload
payload = {
    "model": "gemma3:1b",
    "prompt": final_prompt,
    "stream": True,
    "temperature": 0.1
}

# 4. Send request
response = requests.post(api_url, json=payload, stream=True)

# 5. Read streamed chunks and build the answer
print("\n\nANSWER:\n")
full_answer = ""

if response.status_code == 200:
    for line in response.iter_lines():
        if line:
            data = line.decode('utf-8')
            chunk = json.loads(data)
            if 'response' in chunk:
                token = chunk['response']
                full_answer += token
                print(token, end='', flush=True)
else:
    print(f"Error: {response.status_code} - {response.text}")

# 6. After streaming is done, print references
print("\n\nREFERENCES:\n")
for index, ref in enumerate(references):
    cleaned_ref = ref.strip()
    if cleaned_ref:
        print(f"Reference [{index + 1}]: {cleaned_ref}\n")




ANSWER:

I don't know.

REFERENCES:

Reference [1]: at high velocity”
- What is DevOps? [AWS] -
“A compound of development (Dev) and operations (Ops), DevOps is the 
union of people, process, and technology to continually provide value to 
customers.” 
- What is DevOps? [Azure] -
DevOps allows evolving and improving products at a faster pace than businesses 
using traditional software development and infrastructure management processes. 
This speed allows businesses to serve their customers better and compete effectively.Reduce Organizational

Reference [2]: investors) without understanding 
the complexities behind 
development and operations
Act 04 - Dev and Ops at war
"It worked on my machine" 
phenomenon 
"Destructive downward spiral in IT" - Gene Kim
How can we 
overcome 
these issues?“DevOps is the combination of cultural philosophies, practices, and tools 
that increases an organization’s ability to deliver applications and services 
at high velocity”
- What is DevOps? [AWS] -
